Documentation home: https://github.com/engrit-illinois/misc-handy-powershell-examples
By mseng3
Converted to a Jupyter Notebook by han44

To use this, you'll need a Jupyter Notebook editor. An easy one to get up and working is VSCode:
1. Open Powershell as admin
2. Run `winget install Microsoft.DotNet.SDK.7`
3. Download/Install VSCode e.g. `winget install Microsoft.VisualStudioCode`
4. Install the Polyglot Notebooks extension in Visual Studio Code

Additionally, some of the examples below only really make sense in an interactive Powershell 7 session, so make sure to install Powershell 7!
1. Open Powershell as admin
2. Run `winget install Microsoft.Powershell`
3. Open the new Powershell 7 terminal! Or even better, use the `Terminal` app

Additionally, some examples that interact with MECM can *only* be done in a native Powershell session, so don't run those here!

Do something remotely on multiple computers, in parallel...
https://devblogs.microsoft.com/powershell/powershell-foreach-object-parallel-feature/
Note the parallel functionality of ForEach-Object requires PowerShell 7
To do them NOT in parallel, just remove the `-ThrottleLimit` and `-Parallel` parameters from `ForEach-Object`

In [ ]:
# ...based on specific given computer names
$compNames = "MEL-1001-10","KH-105-03","KH-107-03","EH-406B8-28" # etc., etc.
$compNames

In [ ]:
# ...based on an AD name query
$comps = Get-ADComputer -Filter { Name -like "gelib-4c-*" } # to get all like-named computers
$comps

In [ ]:
# ...based on one or more AD name queries
$queries = "gelib-4e-*","dcl-l426-*","mel-1001-01"
$searchbase="OU=Engineering,OU=Urbana,DC=ad,DC=uillinois,DC=edu"
$comps = @()
foreach($query in @($queries)) {
	$results = Get-ADComputer -SearchBase $searchbase -Filter "name -like `"$query`"" -Properties *
	$comps += @($results)
}
# Return just the name, for the sake of this example
$comps.Name

In [ ]:
# ...based on direct membership rules of an MECM collection
Prep-MECM
$compNames = Get-CMCollectionDirectMembershipRule -CollectionName "UIUC-ENGR-IS mseng3 Test VMs (direct membership)" | Select -ExpandProperty RuleName | Sort

In [ ]:
# ...based on sequentially-named computers (i.e. a lab)
$lab = "ECEB-3024"
$nums = @(4,5,7,11,14)
$comps = @()
$nums | ForEach-Object {
    $num = ([string]$_).PadLeft(2,"0")
    $comps += "$lab-$($num)"
}
$comps

In [ ]:
# Doing the thing
# if the members of $comps are AD or otherwise PowerShell object, instead of just strings
if($comps.ObjectClass -eq 'computer'){
    $compNames = $comps | Select -ExpandProperty "Name" 
}
$compNames | ForEach-Object -ThrottleLimit 15 -Parallel {
    Write-Host "Processing $_..."
    if(Test-Connection $_ -Quiet -Count 1){                 # Adding a ping test to avoid timeout
        Invoke-Command -ComputerName $_ -ScriptBlock {
            # Do stuff here
        }
    }
}

Start a new, elevated PowerShell process in the current directory:
Useful because Windows does not allow Powershell to  elevate an existing session
https://superuser.com/a/1256947/137753

In [10]:
# Powershell 5
Start-Process powershell -Verb runas -ArgumentList "-NoExit -c cd '$pwd'"

In [ ]:
# Powershell 7
Start-Process pwsh -Verb runas -ArgumentList "-NoExit -c cd '$pwd'"